In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas
from scraper import guardian_dataset
import numpy

# First Steps
In order to summmarize, we must first have a working way to group a corpus into phrases. Thats where this comes in

In [2]:
def stack(array, second_dim,  cat_dim, stack_dim):
    max_idx = max(array, key=lambda i: i.shape[0]).shape[0]
    max_idx
    out = []
    for article in array:
        remaining = max_idx - article.shape[0]
        zeros_padding = torch.zeros((remaining, second_dim))
        padded = torch.cat((article, zeros_padding),cat_dim)
        out.append(padded)
    return torch.stack(out, dim = stack_dim)

I would like to thank the guardian for not having a paywall, so I was able to scrape the entire site and pull a bunch of articles

In [2]:
w2v = guardian_dataset.w2v

NameError: name 'guardian_dataset' is not defined

In [4]:
#Fairly Simple classifier
class classifier(nn.Module):

    def __init__(self):
        super(classifier, self).__init__()
        self.lstm1 = nn.LSTM(300, 1)        
    def forward(self, x):
        x = self.lstm1(x)
        return x

net = classifier()
print(net)

classifier(
  (lstm1): LSTM(300, 1)
  (lin1): Linear(in_features=300, out_features=1, bias=True)
  (sig1): LogSigmoid()
)


In [5]:
articles = ['3', '10']
t_x = list()
t_y = list()

for a in articles:
    with open(f"phrase_dataset/article{a}.csv", 'r') as f:
        df = pandas.read_csv(f)
    x_a = list()
    y_a = list()
    for i in range(len(df)):
        word = df.iloc[i, 1]
        res = df.iloc[i, 2]
        x_a.append(torch.tensor(w2v[word]).requires_grad_(True))
        y_a.append(torch.tensor(numpy.asarray(res, dtype='float32')))
    t_x.append(torch.stack(x_a))
    t_y.append(torch.stack(y_a))




<ipython-input-5-3030377a004a>:13: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629434582/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  x_a.append(torch.tensor(w2v[word]).requires_grad_(True))


In [6]:
max_idx = max(t_x, key=lambda i: i.shape[0]).shape[0]
max_idx
out = []
for article in t_x:
    remaining = max_idx - article.shape[0]
    zeros_padding = torch.zeros((remaining, 300))
    padded = torch.cat((article, zeros_padding), 0)
    out.append(padded)
t_x = torch.stack(out, dim = -2)

In [8]:
max_idx = max(t_y, key=lambda i: i.shape[0]).shape[0]
max_idx
out = []
for article in t_y:
    remaining = max_idx - article.shape[0]
    zeros_padding = torch.zeros((remaining))
    padded = torch.cat((article, zeros_padding), 0 )
    out.append(padded)
t_y = torch.stack(out, dim = 0)

In [9]:
z = net.forward(t_x)

In [10]:
epochs = 300

In [11]:
loss_fxn = nn.MSELoss()
optim = torch.optim.Adam(net.parameters(), lr = 0.1)

In [1]:
for epoch in range(epochs):
    optim.zero_grad()
    z = net.forward(t_x)
    loss = loss_fxn(z, t_y)
    print(loss)
    loss.backward()
    optimizer.step()

NameError: name 'epochs' is not defined

# Although not in this notebook, I have done a decent amount of testing with this network and it turns out to work decently well, enough for these purposes at least